In [1]:
import pandas as pd 
import numpy as np 
import distance

In [102]:
df_products = pd.read_csv("HOME_MISSION_SHARE/Recomendations/products_data_fr_fr_2017-11-14_150627.187511.csv", \
                          delimiter=";", encoding="latin1")
df_alerts = pd.read_csv("HOME_MISSION_SHARE/Recomendations/alerts_fr_fr_2017-11-14_151447.572712.csv", delimiter=";", encoding="latin1")

In [103]:
df_products.head(50)

,title_fr_fr,id,description_fr_fr
0,HTC Windows 8S 4 Go - Noir - Bouygues,222,"\tSmartphone sous Android 2.1, le WildFire vie..."
1,Nokia Lumia 1020 32 Go - Blanc - SFR,228,Compatible 4G - Windows Phone 8 - Ecran AMOLED...
2,iPhone 4 16 Go - Noir - Débloqué,1178,<p><b>iPhone 4 16 Go - Noir Débloqué</b></p>\...
3,iPhone 4 16 Go - Blanc - Débloqué,1179,<strong> Le mobile sera livré sous le dernier ...
4,iPhone 4 32 Go - Noir - Débloqué,1180,<p><b>iPhone 4 32 Go - Noir - Débloqué</b></p>...
5,iPhone 4 8 Go - Noir - Débloqué,1181,<b>iPhone 4 8 Go - Noir - Débloqué</b></br>\n<...
6,iPhone 4S 16 Go - Blanc - Débloqué,1182,<p><b>iPhone 4S 16 Go - Blanc Débloqué</b></p...
7,iPhone 4S 16 Go - Noir - Débloqué,1183,<strong>iPhone 4S 16 Go - Noir - Débloqué</str...
8,iPhone 4S 8 Go - Noir - Débloqué,1184,"iPhone 4S 8 Go Noir\n\nEn octobre 2011, Apple ..."
9,iPhone 5 16 Go - Noir - Débloqué,1185,<strong>iPhone 5 16 Go Noir</strong>\n\nLa fir...


In [104]:
df_alerts.head(100)

,pk,product,criterias,country_code,creation_date
0,30254,IPAD MINI 4 32 GO WIFI + 4g Or Débloqué,NaN,fr_fr,2017-11-14 13:09:03+00:00
1,30252,EXPRESSO À CAPSULES Dolce Gusto Krups Yy1786fd,NaN,fr_fr,2017-11-14 12:18:30+00:00
2,30244,ACER SWIFT Sf314-51-39zj 14 Core I3-6006u 2 Gh...,NaN,fr_fr,2017-11-14 11:43:47+00:00
3,30226,IPAD 4 128 GO WIFI NOIR,NaN,fr_fr,2017-11-14 07:52:01+00:00
4,30217,COMPTEUR DE FANS Facebook 5 Chiffres,NaN,fr_fr,2017-11-14 06:27:33+00:00
5,30216,ASUS ZENFONE 2 SELFIE 32 Go Bleu Débloqué,NaN,fr_fr,2017-11-13 23:34:03+00:00
6,30207,DORO 8031 8 GO NOIR Débloqué,NaN,fr_fr,2017-11-13 22:02:28+00:00
7,30199,VENTILATEUR DYSON Cool Am07,NaN,fr_fr,2017-11-13 20:39:24+00:00
8,30198,LG G6 32 GO NOIR Débloqué,NaN,fr_fr,2017-11-13 20:42:47+00:00
9,30197,"IPAD MINI 4 7,9'' 32 GO Wifi Argent",NaN,fr_fr,2017-11-13 20:32:07+00:00


In [98]:
df_products.head()

def words(x, word):
    #Le premier mot est toujours le plus important dans les alertes
    if word.split()[0] in x:
        x = x[x.find(word.split()[0]):]
        #print(x, word)
        return distance.levenshtein(x[:len(word)+1].upper(), word.upper())
    else :
        return 999
prod = "iphone7"
df_products["leveinshtein"] = df_products["title_fr_fr"].map(lambda x : words(x.upper().replace('-',''), prod.upper().replace('-','')))
print(df_products[df_products["leveinshtein"] == df_products["leveinshtein"].min() ].iloc[0,0])


KeyError: 'title_fr_fr'

In [99]:
for prod in df_alerts["product"]:
    df_products["leveinshtein"] = df_products["title_es_es"].map(lambda x : words(x.upper().replace('-',''), prod.upper().replace('-','')))
    print(df_products[df_products["leveinshtein"] == df_products["leveinshtein"].min() ].iloc[0,0])

MacBook Pro 13 Core i5 2.5 GHz  - HDD 750 GB - RAM 8 GB - AZERTY
TV LED Philips 43PFT4001/05
Lenovo Moto G4 Plus - 16 GB - Negro - Libre
iPhone 8 64 GB - Gris espacial - Libre
BlackBerry Bold 9900 8 GB - Negro - desbloqueado
iPhone 6S Plus 16 GB - Gris espacial - Libre
iPhone 6S Plus 16 GB - Gris espacial - Libre
iPhone 6 Plus 64 GB - Plata - Libre
iPhone 6 Plus 64 GB - Plata - Libre
iPhone 6S Plus 16 GB - Gris espacial - Libre
iPhone 6S Plus 16 GB - Gris espacial - Libre
iPhone 6S Plus 16 GB - Gris espacial - Libre
iPhone 6S 64 GB - Rosa - Libre
iMac 21,5 Core i5 2.5 GHz  - HDD 500 GB - RAM 4 GB - AZERTY
iPhone 6S 128 GB - Oro - Libre
Microsoft Lumia 950 32 GB - Blanco - Libre
Sony SmartWatch 3 SWR50 - Correa Negra
Radiador Delonghi KG010920R
TV LED 123 cm LG 49LH570V
iPhone 6S 128 GB - Oro - Libre


In [90]:
df_products[df_products["leveinshtein"] == df_products["leveinshtein"].min() ]

,title_fr_fr,id,description_fr_fr,leveinshtein
1843,iPad 4 128 Go - Wifi - Noir,3828,<strong>iPad 4 128 Go - Wifi - Noir</strong>\n...,3


In [28]:
df_alerts.head()

,pk,product,criterias,country_code,creation_date
0,30254,IPAD MINI 4 32 GO WIFI + 4g Or Débloqué,NaN,fr_fr,2017-11-14 13:09:03+00:00
1,30252,EXPRESSO À CAPSULES Dolce Gusto Krups Yy1786fd,NaN,fr_fr,2017-11-14 12:18:30+00:00
2,30244,ACER SWIFT Sf314-51-39zj 14 Core I3-6006u 2 Gh...,NaN,fr_fr,2017-11-14 11:43:47+00:00
3,30226,IPAD 4 128 GO WIFI NOIR,NaN,fr_fr,2017-11-14 07:52:01+00:00
4,30217,COMPTEUR DE FANS Facebook 5 Chiffres,NaN,fr_fr,2017-11-14 06:27:33+00:00


In [16]:
score = {}
for product in df_products["title_fr_fr"]: 
    
for item in df_alerts["product"][0].split(" "): 
    

['IPAD', 'MINI', '4', '32', 'GO', 'WIFI', '+', '4g', 'Or', 'Débloqué']